In [ ]:

   !pip install pytesseract
   !pip install tesserocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 30.8 MB/s eta 0:00:00


In [ ]:
!apt-get install -y tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,632 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123614 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [29]:
!apt-get install -y tesseract-ocr

import os
import fitz  # PyMuPDF
import pytesseract
import cv2
import io
from PIL import Image, ImageFile
from colorama import Fore
import platform

ImageFile.LOAD_TRUNCATED_IMAGES = True
scanned_text = ""
input_tesseract_path = r"tesseract"
dir_names = ["images", "output_txt"]

def get_user_input():
    global input_tesseract_path

    if platform.system() == "Windows":
        input_tesseract_path = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

    user_input = r"/content/image-based-pdf-sample.pdf"  # Change this to your PDF path
    print(Fore.GREEN + f"[!] Using PDF file: {user_input}" + Fore.RESET)

    if not os.path.isfile(user_input):
        print(Fore.RED + "[X] Please provide a valid path to a file." + Fore.RESET)
    else:
        extract_images(user_input)
        recognize_text()

def extract_images(file_path):
    global dir_names

    pdf_file = fitz.open(file_path)

    for dir_name in dir_names:
        os.makedirs(dir_name, exist_ok=True)
        print(Fore.GREEN + f"[!] Directory {dir_name} created or already exists." + Fore.RESET)

    for page_index in range(len(pdf_file)):
        page = pdf_file[page_index]
        image_list = page.get_images()

        if image_list:
            print(Fore.GREEN + f"[+] Found {len(image_list)} images on page {page_index}." + Fore.RESET)
            for img_index, img in enumerate(image_list, start=1):
                xref = img[0]
                base_image = pdf_file.extract_image(xref)
                image_bytes = base_image["image"]
                image_ext = base_image["ext"]

                image = Image.open(io.BytesIO(image_bytes))
                image.save(open(f"images/image{page_index + 1}_{img_index}.{image_ext}", "wb"))
                print(Fore.GREEN + f"[+] Saved image: images/image{page_index + 1}_{img_index}.{image_ext}" + Fore.RESET)

def recognize_text():
    global scanned_text, input_tesseract_path

    pytesseract.pytesseract.tesseract_cmd = input_tesseract_path

    for img_filename in os.listdir('images'):
        image = cv2.imread(f'images/{img_filename}')

        print(Fore.YELLOW + f"[.] Scanning text from {img_filename}..." + Fore.RESET)
        text = pytesseract.image_to_string(image, lang='eng')  # Change 'eng' to 'spa' for Spanish or other languages

        scanned_text += text + "\n"
        print(Fore.GREEN + "[!] Finished scanning text." + Fore.RESET)

    print(Fore.CYAN + "[.] Writing txtResult.txt..." + Fore.RESET)
    with open(f"{dir_names[1]}/txtResult.txt", "w") as file_txt:
        file_txt.write(scanned_text)
    print(Fore.GREEN + "[!] File written." + Fore.RESET)

get_user_input()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
[!] Using PDF file: /content/image-based-pdf-sample.pdf
[!] Directory images created or already exists.
[!] Directory output_txt created or already exists.
[+] Found 1 images on page 0.
[+] Saved image: images/image1_1.jpeg
[.] Scanning text from image1_1.jpeg...
[!] Finished scanning text.
[.] Writing txtResult.txt...
[!] File written.


In [27]:
!chmod +x /content/tesseract.exe